# Predicciones con RandomForest

In [2]:
import numpy as np
import pandas as pd
import pandas.io

import matplotlib.pyplot as plt

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

import pickle

In [9]:
filename = 'finalized_model_fourier.sav'

loaded_model = pickle.load(open(filename, 'rb'))

In [10]:
prueba1 = pd.read_csv('data_fourier_aux.csv')
prueba1

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-1.628756,1414.572244,22.675694,2688.964844,1012.104539,133.928571,0.147794,0.455576,0.855517,1.217035,...,0.227303,0.221763,0.201087,0.402851,0.266518,0.188775,0.448843,1.217035,0.855517,Aspiradora
1,-1.506035,1381.930099,22.745829,2651.977539,1012.829229,133.928571,0.143885,1.337814,0.821697,2.255564,...,0.072751,0.030290,0.495613,0.288902,0.457856,0.366302,0.719223,2.255564,0.821697,Aspiradora
2,-10.171223,1899.130782,22.926630,3330.078125,1053.184961,133.928571,0.166528,0.146833,0.296734,1.424936,...,0.606584,1.065522,0.388100,1.458757,0.561210,0.724556,1.026013,1.424936,0.296734,Aspiradora
3,-6.319030,2068.097241,19.717858,3147.338867,1034.022848,133.928571,0.063924,0.217815,0.299719,0.650947,...,0.303021,0.250098,0.341444,0.131757,0.194542,0.498210,0.393564,0.650947,0.299719,Licuadora
4,-4.020776,1735.879747,32.736242,3238.281250,1069.616398,120.000000,0.178995,0.150675,0.519349,3.256600,...,0.224028,0.152183,0.879043,1.191371,1.137622,1.196225,0.238380,3.256600,0.519349,Aspiradora
5,-9.823275,1879.801904,23.219836,3299.194336,1049.653854,117.187500,0.170869,0.227055,0.128607,2.615066,...,0.735094,0.116490,0.625228,1.052007,0.477034,1.126863,0.340256,2.615066,0.128607,Aspiradora
6,-9.348661,1865.229316,19.865615,3089.965820,1048.262371,133.928571,0.037031,0.138370,0.923939,0.571617,...,0.434972,0.470833,0.089985,0.096192,0.339012,0.609834,0.591655,0.571617,0.923939,Licuadora
7,-9.351369,1877.062816,23.002090,3314.575195,1059.631430,133.928571,0.179560,0.283336,0.287780,3.256087,...,0.488734,0.544311,0.853358,1.796916,0.662185,1.254831,0.540638,3.256087,0.287780,Aspiradora
8,-7.407472,2064.987327,19.966459,3147.949219,1035.769594,133.928571,0.061652,0.094789,0.557706,0.315581,...,0.718631,0.223358,0.207510,0.170977,0.195823,0.325118,0.467143,0.315581,0.557706,Licuadora
9,-10.974715,1885.805808,23.011807,3298.828125,1038.360506,133.928571,0.160421,0.000183,0.000158,0.000110,...,0.000151,0.000164,0.000057,0.000118,0.000064,0.000178,0.000128,0.000110,0.000158,Aspiradora


In [11]:
classes = ['Afeitadora', 'Aire_acondicionado', 'Alarma', 'Aspiradora', 'Cena', 'Ducha', 'Grifo', 'Lavadora', 
           'Licuadora', 'Microondas', 'Secador_pelo', 'Secadora', 'Silencio', 'Tecleo', 'Telefono', 'Timbre']

X_new = prueba1.loc[:, prueba1.columns != 'momento']
y_new = loaded_model.predict(X_new)
y_new2 = loaded_model.predict_proba(X_new)

for i in range(len(X_new)):
    predicted_list = y_new[i].tolist()
    if 1 in predicted_list:
        predicted_class = classes[y_new[i].tolist().index(1)]
        predicted_probability = y_new2[i].tolist()[y_new[i].tolist().index(1)]
        print('Para este audio el valor predicho es {} y la probabilidad de haber acertado es del {:.2f}%.'.format(predicted_class, predicted_probability*100))
    else:
        print('El sonido analizado no se encuentra entre los sonidos más comunes de una casa.')

Para este audio el valor predicho es Aspiradora y la probabilidad de haber acertado es del 51.67%.
Para este audio el valor predicho es Aspiradora y la probabilidad de haber acertado es del 53.33%.
Para este audio el valor predicho es Aspiradora y la probabilidad de haber acertado es del 71.67%.
Para este audio el valor predicho es Grifo y la probabilidad de haber acertado es del 56.67%.
Para este audio el valor predicho es Aspiradora y la probabilidad de haber acertado es del 53.33%.
Para este audio el valor predicho es Aspiradora y la probabilidad de haber acertado es del 66.67%.
Para este audio el valor predicho es Ducha y la probabilidad de haber acertado es del 61.67%.
Para este audio el valor predicho es Aspiradora y la probabilidad de haber acertado es del 63.33%.
Para este audio el valor predicho es Grifo y la probabilidad de haber acertado es del 65.00%.
Para este audio el valor predicho es Alarma y la probabilidad de haber acertado es del 55.00%.
Para este audio el valor pred